In [1]:
import utils, RKHS_DAGMA_extractj
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
from CausalDisco.analytics import r2_sortability, var_sortability
from CausalDisco.baselines import r2_sort_regress, var_sort_regress
from cdt.metrics import SID, SHD
import seaborn as sns
import cdt
import multiprocessing
import threading
import json

Detecting 1 CUDA device(s).


In [2]:
sns.set_context("paper")

torch.set_default_dtype(torch.float64)

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)

# utils.set_random_seed(1)
# torch.manual_seed(1)

cdt.SETTINGS.rpath= 'C:\Program Files\R\R-4.3.3\\bin\Rscript'

num_cores = multiprocessing.cpu_count()

# Print the number of CPU cores
# print("Number of CPU cores:", num_cores)
# print(torch.__version__)

# Do RKHS_DAGMA Discovery

In [3]:
def run_model(lambda1, tau, X, device, B_true, gamma, T=6, lr = 0.03):
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    results = {}
    
    X = X.to(device)
    eq_model = RKHS_DAGMA_extractj.DagmaRKHS(X, gamma = gamma).to(device)
    model = RKHS_DAGMA_extractj.DagmaRKHS_nonlinear(eq_model)
    
    x_est_start = eq_model.forward()
    start_mse = eq_model.mse(x_est_start).detach().cpu().numpy()
    time_start = time.time()
    W_est_no_thresh, output = model.fit(X, lambda1=lambda1, tau=tau, T = T, mu_init = 1.0, lr=lr, w_threshold=0.0)
    time_end = time.time()

    thresh_values = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
    for thresh in thresh_values:
        try:
            W_est_dagma = abs(W_est_no_thresh) * (abs(W_est_no_thresh) > thresh)
            acc_dagma = utils.count_accuracy(B_true, W_est_dagma != 0)
            valid = "yes"
        #sid_dagma = SID(B_true, W_est_dagma != 0).item()
        except Exception as e:
            W_est_dagma = results_var_sort_regress
            acc_dagma = utils.count_accuracy(B_true, W_est_dagma != 0)
            valid = "no"
            
        diff_dagma = np.linalg.norm(W_est_dagma - abs(B_true))
        x_est = eq_model.forward()
        mse_dagma = eq_model.mse(x_est).detach().cpu().numpy()
        W_est = eq_model.fc1_to_adj()
        h_val = eq_model.h_func(W_est, s=1).detach().cpu().numpy()
        #parameters = eq_model.get_parameters()
        W_est_dagma = W_est_dagma.tolist()
        B = B_true.tolist()
        key = f'lambda_{lambda1}_tau_{tau}_thresh_{thresh}_gamma_{gamma}'
        results[key] = {
            'SHD': acc_dagma['shd'],
            'TPR': acc_dagma['tpr'],
            'Time Elapsed': time_end - time_start,
            'F1': acc_dagma['f1'],
            'diff': diff_dagma,
            'mse': mse_dagma.item(),
            'valid': valid,
            'h_val': h_val.item(),
            'start mse': start_mse,
            #'W_est_dagma': W_est_dagma,
            #'parameters': parameters,
            #'eq_model': eq_model,
            #'W_est_no_thresh': W_est_no_thresh,
            #'B_true': B
        }

    torch.cuda.empty_cache()

    # filename = f'model_results_lambda_{lambda1}_tau_{tau}.txt'

    # with open(filename, 'w') as file:
    #     json.dump(results, file, indent=4) 

    return results

In [4]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print('device:', device)
torch.set_default_device(device)

device: cuda


## d = 10

In [9]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 10, 40, 'ER', 'gp-add' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [10]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 4, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_4': {'SHD': 26, 'TPR': 0.375, 'Time Elapsed': 209.1312313079834, 'F1': 0.5172413793103448, 'diff': 6.568632232423043, 'mse': 7.909538475310949, 'valid': 'no', 'h_val': 1.6008679725677286e-06, 'start mse': array(25.30947959)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_4': {'SHD': 10, 'TPR': 0.875, 'Time Elapsed': 209.1312313079834, 'F1': 0.823529411764706, 'diff': 5.13587751273438, 'mse': 7.909538475310949, 'valid': 'yes', 'h_val': 1.6008679725677286e-06, 'start mse': array(25.30947959)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_4': {'SHD': 16, 'TPR': 0.725, 'Time Elapsed': 209.1312313079834, 'F1': 0.7341772151898733, 'diff': 5.165622246521774, 'mse': 7.909538475310949, 'valid': 'yes', 'h_val': 1.6008679725677286e-06, 'start mse': array(25.30947959)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_4': {'SHD': 18, 'TPR': 0.675, 'Time Elapsed': 209.1312313079834, 'F1': 0.7012987012987014, 'diff': 5.194646505816487, 'mse': 7.909538475310949, 'vali

In [13]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 10, 40, 'ER', 'gp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [14]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 4, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_4': {'SHD': 40, 'TPR': 0.025, 'Time Elapsed': 201.6461842060089, 'F1': 0.047619047619047616, 'diff': 6.269630453113852, 'mse': 6.361944944035654, 'valid': 'no', 'h_val': 8.547414355054411e-07, 'start mse': array(8.48563398)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_4': {'SHD': 17, 'TPR': 0.7, 'Time Elapsed': 201.6461842060089, 'F1': 0.6588235294117647, 'diff': 5.763713175934845, 'mse': 6.361944944035654, 'valid': 'yes', 'h_val': 8.547414355054411e-07, 'start mse': array(8.48563398)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_4': {'SHD': 18, 'TPR': 0.625, 'Time Elapsed': 201.6461842060089, 'F1': 0.6666666666666666, 'diff': 5.772436609811687, 'mse': 6.361944944035654, 'valid': 'yes', 'h_val': 8.547414355054411e-07, 'start mse': array(8.48563398)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_4': {'SHD': 25, 'TPR': 0.425, 'Time Elapsed': 201.6461842060089, 'F1': 0.5573770491803278, 'diff': 5.869786957683098, 'mse': 6.361944944035654, 'valid': 

In [15]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 10, 40, 'ER', 'mlp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [16]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 4, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_4': {'SHD': 11, 'TPR': 0.85, 'Time Elapsed': 259.5287575721741, 'F1': 0.7999999999999998, 'diff': 4.715235982059768, 'mse': 21.08543219133373, 'valid': 'yes', 'h_val': 8.829247121200368e-07, 'start mse': array(233.45203392)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_4': {'SHD': 11, 'TPR': 0.85, 'Time Elapsed': 259.5287575721741, 'F1': 0.7999999999999998, 'diff': 4.715235982059768, 'mse': 21.08543219133373, 'valid': 'yes', 'h_val': 8.829247121200368e-07, 'start mse': array(233.45203392)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_4': {'SHD': 13, 'TPR': 0.8, 'Time Elapsed': 259.5287575721741, 'F1': 0.7710843373493975, 'diff': 4.72934054134763, 'mse': 21.08543219133373, 'valid': 'yes', 'h_val': 8.829247121200368e-07, 'start mse': array(233.45203392)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_4': {'SHD': 14, 'TPR': 0.775, 'Time Elapsed': 259.5287575721741, 'F1': 0.7560975609756099, 'diff': 4.740442265928565, 'mse': 21.08543219133373, 'valid'

## d = 20

In [7]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 20, 80, 'ER', 'gp-add' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [8]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 8, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_8': {'SHD': 64, 'TPR': 0.3375, 'Time Elapsed': 349.5824701786041, 'F1': 0.43200000000000005, 'diff': 9.568051707117414, 'mse': 21.088015354556916, 'valid': 'no', 'h_val': 1.1306787610895247e-05, 'start mse': array(45.54741618)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_8': {'SHD': 64, 'TPR': 0.3375, 'Time Elapsed': 349.5824701786041, 'F1': 0.43200000000000005, 'diff': 9.568051707117414, 'mse': 21.088015354556916, 'valid': 'no', 'h_val': 1.1306787610895247e-05, 'start mse': array(45.54741618)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_8': {'SHD': 76, 'TPR': 0.475, 'Time Elapsed': 349.5824701786041, 'F1': 0.4840764331210191, 'diff': 8.420834580843083, 'mse': 21.088015354556916, 'valid': 'yes', 'h_val': 1.1306787610895247e-05, 'start mse': array(45.54741618)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_8': {'SHD': 74, 'TPR': 0.25, 'Time Elapsed': 349.5824701786041, 'F1': 0.3508771929824561, 'diff': 8.558906100809642, 'mse': 21.08801535455691

In [13]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 20, 80, 'ER', 'gp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [14]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 8, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_8': {'SHD': 77, 'TPR': 0.075, 'Time Elapsed': 358.0812156200409, 'F1': 0.1348314606741573, 'diff': 9.018154120229605, 'mse': 13.202065018402735, 'valid': 'no', 'h_val': 2.8511056450403172e-06, 'start mse': array(16.40777158)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_8': {'SHD': 77, 'TPR': 0.075, 'Time Elapsed': 358.0812156200409, 'F1': 0.1348314606741573, 'diff': 9.018154120229605, 'mse': 13.202065018402735, 'valid': 'no', 'h_val': 2.8511056450403172e-06, 'start mse': array(16.40777158)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_8': {'SHD': 81, 'TPR': 0.25, 'Time Elapsed': 358.0812156200409, 'F1': 0.31496062992125984, 'diff': 8.672618224686337, 'mse': 13.202065018402735, 'valid': 'yes', 'h_val': 2.8511056450403172e-06, 'start mse': array(16.40777158)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_8': {'SHD': 76, 'TPR': 0.15, 'Time Elapsed': 358.0812156200409, 'F1': 0.2352941176470588, 'diff': 8.729205286079237, 'mse': 13.202065018402735, '

In [15]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 20, 80, 'ER', 'mlp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [16]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 8, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_8': {'SHD': 89, 'TPR': 0.3125, 'Time Elapsed': 417.28156876564026, 'F1': 0.32258064516129037, 'diff': 11.57340103047146, 'mse': 157.45458406430626, 'valid': 'no', 'h_val': 0.0001286312577131111, 'start mse': array(566.18164828)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_8': {'SHD': 89, 'TPR': 0.3125, 'Time Elapsed': 417.28156876564026, 'F1': 0.32258064516129037, 'diff': 11.57340103047146, 'mse': 157.45458406430626, 'valid': 'no', 'h_val': 0.0001286312577131111, 'start mse': array(566.18164828)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_8': {'SHD': 114, 'TPR': 0.325, 'Time Elapsed': 417.28156876564026, 'F1': 0.2653061224489796, 'diff': 8.885367590658824, 'mse': 157.45458406430626, 'valid': 'yes', 'h_val': 0.0001286312577131111, 'start mse': array(566.18164828)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_8': {'SHD': 98, 'TPR': 0.1375, 'Time Elapsed': 417.28156876564026, 'F1': 0.16176470588235295, 'diff': 8.975071126464346, 'mse': 157.45458

## d = 30

In [17]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 30, 120, 'ER', 'gp-add' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [18]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 12, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_12': {'SHD': 103, 'TPR': 0.35, 'Time Elapsed': 753.0553665161133, 'F1': 0.43523316062176165, 'diff': 11.229162064543418, 'mse': 36.44354150093772, 'valid': 'no', 'h_val': 2.8843992515349805e-05, 'start mse': array(81.20259934)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_12': {'SHD': 103, 'TPR': 0.35, 'Time Elapsed': 753.0553665161133, 'F1': 0.43523316062176165, 'diff': 11.229162064543418, 'mse': 36.44354150093772, 'valid': 'no', 'h_val': 2.8843992515349805e-05, 'start mse': array(81.20259934)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_12': {'SHD': 137, 'TPR': 0.35833333333333334, 'Time Elapsed': 753.0553665161133, 'F1': 0.3659574468085106, 'diff': 10.558407680107765, 'mse': 36.44354150093772, 'valid': 'yes', 'h_val': 2.8843992515349805e-05, 'start mse': array(81.20259934)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_12': {'SHD': 111, 'TPR': 0.175, 'Time Elapsed': 753.0553665161133, 'F1': 0.2675159235668789, 'diff': 10.68296720381211, 'mse'

In [19]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 30, 120, 'ER', 'gp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [20]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 12, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_12': {'SHD': 125, 'TPR': 0.10833333333333334, 'Time Elapsed': 962.8019144535065, 'F1': 0.17218543046357618, 'diff': 10.92090148941441, 'mse': 21.08564965307828, 'valid': 'no', 'h_val': 4.613701580805549e-06, 'start mse': array(28.4289835)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_12': {'SHD': 125, 'TPR': 0.10833333333333334, 'Time Elapsed': 962.8019144535065, 'F1': 0.17218543046357618, 'diff': 10.92090148941441, 'mse': 21.08564965307828, 'valid': 'no', 'h_val': 4.613701580805549e-06, 'start mse': array(28.4289835)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_12': {'SHD': 167, 'TPR': 0.20833333333333334, 'Time Elapsed': 962.8019144535065, 'F1': 0.21186440677966104, 'diff': 10.716292622956653, 'mse': 21.08564965307828, 'valid': 'yes', 'h_val': 4.613701580805549e-06, 'start mse': array(28.4289835)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_12': {'SHD': 133, 'TPR': 0.14166666666666666, 'Time Elapsed': 962.8019144535065, 'F1': 0.1954022988505

In [5]:
utils.set_random_seed(0)
torch.manual_seed(0)
n, d, s0, graph_type, sem_type = 100, 30, 120, 'ER', 'mlp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [6]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 10, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_10': {'SHD': 136, 'TPR': 0.38333333333333336, 'Time Elapsed': 1151.4162907600403, 'F1': 0.3739837398373984, 'diff': 14.821410204881552, 'mse': 337.58798459722254, 'valid': 'no', 'h_val': 0.0003649691711161557, 'start mse': array(1327.67953455)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_10': {'SHD': 136, 'TPR': 0.38333333333333336, 'Time Elapsed': 1151.4162907600403, 'F1': 0.3739837398373984, 'diff': 14.821410204881552, 'mse': 337.58798459722254, 'valid': 'no', 'h_val': 0.0003649691711161557, 'start mse': array(1327.67953455)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_10': {'SHD': 265, 'TPR': 0.19166666666666668, 'Time Elapsed': 1151.4162907600403, 'F1': 0.12777777777777777, 'diff': 11.130906165532679, 'mse': 337.58798459722254, 'valid': 'yes', 'h_val': 0.0003649691711161557, 'start mse': array(1327.67953455)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_10': {'SHD': 194, 'TPR': 0.05, 'Time Elapsed': 1151.4162907600403, 'F1': 0.052631578947

In [7]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 12, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_12': {'SHD': 136, 'TPR': 0.38333333333333336, 'Time Elapsed': 1789.4837696552277, 'F1': 0.3739837398373984, 'diff': 14.821410204881552, 'mse': 30.10209920164627, 'valid': 'no', 'h_val': -9.619390894943791e-09, 'start mse': array(1327.67953455)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_12': {'SHD': 136, 'TPR': 0.38333333333333336, 'Time Elapsed': 1789.4837696552277, 'F1': 0.3739837398373984, 'diff': 14.821410204881552, 'mse': 30.10209920164627, 'valid': 'no', 'h_val': -9.619390894943791e-09, 'start mse': array(1327.67953455)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_12': {'SHD': 136, 'TPR': 0.38333333333333336, 'Time Elapsed': 1789.4837696552277, 'F1': 0.3739837398373984, 'diff': 14.821410204881552, 'mse': 30.10209920164627, 'valid': 'no', 'h_val': -9.619390894943791e-09, 'start mse': array(1327.67953455)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_12': {'SHD': 136, 'TPR': 0.38333333333333336, 'Time Elapsed': 1789.4837696552277, 'F1': 0

In [5]:
utils.set_random_seed(1)
torch.manual_seed(1)
n, d, s0, graph_type, sem_type = 100, 30, 120, 'ER', 'mlp' # s0 = expected number of edges
B_true = utils.simulate_dag(d, s0, graph_type) # [d, d] binary adj matrix of DAG
X = utils.simulate_nonlinear_sem(B_true, n, sem_type) # [n, d] sample matrix
results_var_sort_regress = var_sort_regress(X)
X = torch.from_numpy(X)

In [6]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 12, T=6, lr= 0.03)
print(results)

  0%|          | 0/33000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 798.5386590957642, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 233.04612817601657, 'valid': 'no', 'h_val': 0.0003850311280564261, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 798.5386590957642, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 233.04612817601657, 'valid': 'no', 'h_val': 0.0003850311280564261, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 798.5386590957642, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 233.04612817601657, 'valid': 'no', 'h_val': 0.0003850311280564261, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_12': {'SHD': 138, 'TPR': 0.10833333333333334, 'Time Elapsed': 798.5386590957642, 'F1': 0.155688

In [7]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 12, T=7, lr= 0.03)
print(results)

  0%|          | 0/38000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 1084.9773297309875, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 304.99887980854004, 'valid': 'no', 'h_val': 1.86184792041225e-05, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 1084.9773297309875, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 304.99887980854004, 'valid': 'no', 'h_val': 1.86184792041225e-05, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_12': {'SHD': 157, 'TPR': 0.19166666666666668, 'Time Elapsed': 1084.9773297309875, 'F1': 0.21698113207547173, 'diff': 10.734847202099852, 'mse': 304.99887980854004, 'valid': 'yes', 'h_val': 1.86184792041225e-05, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_12': {'SHD': 140, 'TPR': 0.125, 'Time Elapsed': 1084.9773297309875, 'F1': 0.17341040462427745

In [8]:
results = run_model(lambda1 = 1e-3, tau = 1e-4, X = X, device = device, B_true = B_true, gamma = 12, T=8, lr= 0.03)
print(results)

  0%|          | 0/43000.0 [00:00<?, ?it/s]

{'lambda_0.001_tau_0.0001_thresh_0.005_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 1272.068335533142, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 336.2413550993686, 'valid': 'no', 'h_val': 7.505914315743252e-07, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.01_gamma_12': {'SHD': 108, 'TPR': 0.38333333333333336, 'Time Elapsed': 1272.068335533142, 'F1': 0.4360189573459716, 'diff': 13.678161899222934, 'mse': 336.2413550993686, 'valid': 'no', 'h_val': 7.505914315743252e-07, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.05_gamma_12': {'SHD': 163, 'TPR': 0.225, 'Time Elapsed': 1272.068335533142, 'F1': 0.23893805309734517, 'diff': 10.712562816993469, 'mse': 336.2413550993686, 'valid': 'yes', 'h_val': 7.505914315743252e-07, 'start mse': array(756.02150328)}, 'lambda_0.001_tau_0.0001_thresh_0.1_gamma_12': {'SHD': 147, 'TPR': 0.13333333333333333, 'Time Elapsed': 1272.068335533142, 'F1': 0.17297297297297298, 'd